## Regression _ Logistic Regression
주로 이항분류에 사용됨. 분류 문제에서 사용할 수 있는 가장 간단한 모델<br>
선형 결합을 통해 나온 결과를 토대로 예측하게 된다!!

### with TF-IDF

In [1]:
import os

import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [21]:
DATA_IN_PATH = './data_in/'
TRAIN_CLEAN_DATA = 'train_clean.csv'
DATA_OUT_PATH = './data_out/'

train_data = pd.read_csv(DATA_IN_PATH + TRAIN_CLEAN_DATA) # csv면 comma 기준 아닌가..?


In [3]:
reviews = list(train_data['review'])
sentiments = list(train_data['sentiment'])

In [4]:
vectorizer = TfidfVectorizer(min_df= 0.0, analyzer = 'char', sublinear_tf = True, ngram_range = (1,3), max_features = 5000)

X = vectorizer.fit_transform(reviews)


In [5]:
y = np.array(sentiments)

In [9]:
features = vectorizer.get_feature_names()
features

[' ',
 ' a',
 ' aa',
 ' ab',
 ' ac',
 ' ad',
 ' ae',
 ' af',
 ' ag',
 ' ah',
 ' ai',
 ' ak',
 ' al',
 ' am',
 ' an',
 ' ap',
 ' ar',
 ' as',
 ' at',
 ' au',
 ' av',
 ' aw',
 ' ax',
 ' az',
 ' b',
 ' b ',
 ' ba',
 ' bb',
 ' be',
 ' bi',
 ' bl',
 ' bo',
 ' br',
 ' bu',
 ' by',
 ' c',
 ' c ',
 ' ca',
 ' ce',
 ' cg',
 ' ch',
 ' ci',
 ' cl',
 ' co',
 ' cr',
 ' cu',
 ' cy',
 ' d',
 ' da',
 ' de',
 ' di',
 ' do',
 ' dr',
 ' du',
 ' dv',
 ' dw',
 ' dy',
 ' e',
 ' e ',
 ' ea',
 ' eb',
 ' ec',
 ' ed',
 ' ee',
 ' ef',
 ' eg',
 ' ei',
 ' el',
 ' em',
 ' en',
 ' ep',
 ' eq',
 ' er',
 ' es',
 ' et',
 ' eu',
 ' ev',
 ' ex',
 ' ey',
 ' f',
 ' f ',
 ' fa',
 ' fb',
 ' fe',
 ' fi',
 ' fl',
 ' fo',
 ' fr',
 ' fu',
 ' fx',
 ' g',
 ' g ',
 ' ga',
 ' ge',
 ' gh',
 ' gi',
 ' gl',
 ' go',
 ' gr',
 ' gu',
 ' gw',
 ' gy',
 ' h',
 ' h ',
 ' ha',
 ' hb',
 ' he',
 ' hi',
 ' hm',
 ' ho',
 ' hu',
 ' hy',
 ' i',
 ' ia',
 ' ic',
 ' id',
 ' ig',
 ' ii',
 ' il',
 ' im',
 ' in',
 ' ir',
 ' is',
 ' it',
 ' iv',
 ' j',
 ' j

In [10]:
RANDOM_SEED = 42
TEST_SPLIT = 0.2

In [11]:
# 검증데이터 분리

X_train, X_eval, y_train, y_eval = train_test_split(X, y, test_size=TEST_SPLIT, random_state=RANDOM_SEED)

In [12]:
# 모델 선언 및 학습

lgs = LogisticRegression(class_weight = 'balanced')
lgs.fit(X_train, y_train) # 이걸로 학습 시켜라! lgs 모델을

LogisticRegression(class_weight='balanced')

In [15]:
predicted = lgs.predict(X_eval)

In [14]:
# 검증 데이터로 성능 평가

print("Accuracy: {:.2f}".format(lgs.score(X_eval, y_eval))) # lgs.score 함수를 쓰면 X_eval 자리에 있는 걸 lgs 모델로 예측하고 y_eval과 비교한 accuracy를 알려주나보군!

Accuracy: 0.86


In [16]:
# 검증 데이터에서 만족할만큼 나오면 평가 데이터 적용

TEST_CLEAN_DATA = 'test_clean.csv'

test_data = pd.read_csv(DATA_IN_PATH + TEST_CLEAN_DATA)

In [17]:
testDataVecs = vectorizer.transform(test_data['review'])

In [18]:
test_predicted = lgs.predict(testDataVecs)
print(test_predicted)

[1 0 1 ... 0 1 0]


In [22]:
if not os.path.exists(DATA_OUT_PATH):
    os.makedirs(DATA_OUT_PATH)
    
answer_dataset = pd.DataFrame({'id': test_data['id'], 'sentiment': test_predicted})
answer_dataset.to_csv(DATA_OUT_PATH + 'lgs_tfidf_answer.csv', index= False, quoting=3)

### with word2vec

입력: 단어로 표현된 리스트 (전처리한 넘파이 배열 사용하지 X)


In [23]:
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [24]:
sentences = []
for review in reviews: # reviews 안에 들어있는 건 다 train_data
    sentences.append(review.split())

In [25]:
num_features = 300
min_word_count = 40 # 단어에 대한 최소 빈도 수
num_workers = 4 # 프로세스 개수
context = 10 # 컨텍스트 윈도우 크기
downsampling = 1e-3 # 다운 샘플링 비율

In [28]:
import logging
logging.basicConfig(format = '%(asctime)s : %(levelname)s : %(message)s', \
                   level = logging.INFO)

In [29]:
# word2vec 학습: Word2Vec 객체를 생성해서 실행.
# 이렇게 학습하고 생성된 객체는 model 변수에 할당 (학습을 위한 객체의 인자는 입력할 데이터와 하이퍼파라미터를 순서대로 입력해야 원하는 하이퍼파라미터를 사용해 학습할 수 있음!!!)

from gensim.models import word2vec

model = word2vec.Word2Vec(sentences, workers = num_workers,
                         size = num_features, min_count = min_word_count,
                         window = context, sample = downsampling)

2021-03-24 01:46:00,657 : INFO : collecting all words and their counts
2021-03-24 01:46:00,658 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-03-24 01:46:00,844 : INFO : PROGRESS: at sentence #10000, processed 1205223 words, keeping 51374 word types
2021-03-24 01:46:01,014 : INFO : PROGRESS: at sentence #20000, processed 2396605 words, keeping 67660 word types
2021-03-24 01:46:01,101 : INFO : collected 74065 word types from a corpus of 2988089 raw words and 25000 sentences
2021-03-24 01:46:01,102 : INFO : Loading a fresh vocabulary
2021-03-24 01:46:01,132 : INFO : effective_min_count=40 retains 8160 unique words (11% of original 74065, drops 65905)
2021-03-24 01:46:01,133 : INFO : effective_min_count=40 leaves 2627273 word corpus (87% of original 2988089, drops 360816)
2021-03-24 01:46:01,153 : INFO : deleting the raw counts dictionary of 74065 items
2021-03-24 01:46:01,155 : INFO : sample=0.001 downsamples 30 most-common words
2021-03-24 01:46:01,156 :

In [ ]:
# 모델 저장
# 모델 저장 시 하이퍼파라미터 설정한 내용을 모델 이름에 담으면 참고하기 좋음
# 모델을 저장하면 Word2Vec.load()를 통해 모델을 다시 사용할 수 있음

model_name = "300features_40minwords_10context"
model.save(model_name) # model이 앞에 만든 word2vec 객체

In [34]:
type(model)

gensim.models.word2vec.Word2Vec

In [32]:
# 하나의 리뷰당 하나의 벡터로 만들기. 문장에 있는 모든 단어의 벡터값에 대해 평균을 내는 방법
def get_features(words, model, num_features):
    # 출력 벡터 초기화
    feature_vector = np.zeros((num_features), dtype = np.float32)
    
    num_words = 0
    # 어휘 사전 준비
    index2word_set = set(model.wv.index2word)
    
    for w in words:
        if w in index2word_set:
            num_words += 1
            # 사전에 해당하는 단어에 대해 단어 벡터를 더함
            feature_vector = np.add(feature_vector, model[w]) # model에 인덱스가 있어..?? 그런가 봄.. integer은 안된다고 하고 str만 가능!
            
    feature_vector = np.divide(feature_vector, num_words)
    return feature_vector

In [33]:
# 전체 리뷰에 위의 함수 적용하는 함수 만들기
def get_dataset(reviews, model, num_features):
    dataset = list()
    
    for s in reviews:
        dataset.append(get_features(s, model, num_features))
        
    
    reviewFeatureVecs = np.stack(dataset) # np.stack의 기능은?
    
    return reviewFeatureVecs

In [48]:
test_data_vecs = get_dataset(sentences, model, num_features) # 이제야 학습에 사용할 리뷰별 벡터로 준비됨 (sentences는 train_Data에서 온 아이)

c:\programdata\anaconda3\envs\pr_tensorflow\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [49]:
# 검증 데이터 셋 분리

X = test_data_vecs
y = np.array(sentiments)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = TEST_SPLIT, random_state = RANDOM_SEED)

In [50]:
lgs = LogisticRegression(class_weight = 'balanced')
lgs.fit(X_train, y_train)

c:\programdata\anaconda3\envs\pr_tensorflow\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(class_weight='balanced')

In [51]:
print('Accuracy: {:.2f}'.format(lgs.score(X_test, y_test))) # 검증 데이터로 성능(accuracy) 측정

Accuracy: 0.87


In [52]:
# 제출할 데이터 만들기

TEST_CLEAN_DATA = 'test_clean.csv'
test_data = pd.read_csv(DATA_IN_PATH + TEST_CLEAN_DATA)

test_review = list(test_data['review'])


In [54]:
test_sentences = []
for review in test_review:
    test_sentences.append(review.split())

In [56]:
test_data_vecs = get_dataset(test_sentences, model, num_features)

c:\programdata\anaconda3\envs\pr_tensorflow\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [57]:
DATA_OUT_PATH = './data_out/'
test_predicted = lgs.predict(test_data_vecs)

ids= list(test_data['id'])
answer_dataset = pd.DataFrame({'id': ids, 'sentiment': test_predicted})
answer_dataset.to_csv(DATA_OUT_PATH + 'lgs_w2v_answer.csv', index = False, quoting =3) # quoting 3 뭐가 달라지는 걸까...